Point by point stage movement using set position


In [1]:
from pipython2 import GCSDevice
from pipython2 import gcscommands
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time

In [2]:
#create device object
gcs = GCSDevice('E-727')

In [3]:
print(gcs)

GCSCommands(msgs=GCSMessages(interface=GCSDll(C:\ProgramData\PI\GCSTranslator\PI_GCS2_DLL_x64.dll)), id=-1)


In [4]:
#connect Piezo stage (check COM number and baudrate from PI umove app)
gcs.ConnectRS232(3,460800)


In [5]:
#check connection status and set servo mode ON
#Correct value like  "(c)2015 Physik Instrumente (PI) GmbH & Co. KG, E-727, 0116008043, 13.21.00.08"
print (gcs.qIDN())
gcs.SVO1([1,2,3],[1,1,1])
print(gcs.qSVO1([1,2,3]))  #1 - ON, 0 - OFF. For this script all values should be ON

(c)2015 Physik Instrumente (PI) GmbH & Co. KG, E-727, 0116008043, 13.21.00.08

1=1 
2=1 
3=1



In [6]:
#used basic functions defined here: C:\Anaconda3\Lib\site-packages\PIPython-1.3.9.40-py3.7.egg\pipython\gcscommands.py

def point_by_point_snake_3(gcs,y_points,x_points,dwell_time,
                       delta_dist,offset,rep = 10 ,velocity=50):
    """
        V3 using trigger mode 2
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger on selected channel send signal when stage is on target
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
       velocity: velocity of the stage
       
       Output  is +-2% of registered time and may depend on velocity
    """
    wait_time =dwell_time +delta_dist*1.0/(velocity*1.2)+0.015
    print(wait_time)
    gcs.VEL1([1,2,3],[10000,10000,10000])
    gcs.MOV1([1,2,3],[offset[0]-delta_dist,offset[1],offset[2]-delta_dist])
    input('Press enter to start')
    gcs.VEL1([1,2,3],[velocity,velocity,velocity])
    start =time.time()
    gcs.CTO1(1,[2,3],[1,2])
    gcs.CTO1(2,[2,3],[2,2])
    gcs.CTO1(3,[2,3],[3,2])

    t1=time.time()
    print(0,0,t1-start)
    tnow= time.time()
    for a in range(rep):
        try:
            #Move to initial point
            gcs.MOV1([1,2,3],offset)
            t1=time.time()
            while (time.time()-t1<wait_time):
                continue
            tnow=time.time() 
            print(0,0,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
            for i in range(int(x_points/2)):
                
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])

                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time()
                    print(2*i,j+1)
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1),offset[2]])
                t1=time.time()
                while (time.time()-t1<wait_time):
                    continue
                tnow=time.time() 
                print(2*i+1,j+1)
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time() 
                    print(2*i+1,y_points-j-2)
                if i <int(x_points/2)-1:
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time()
                    print(2*i+2,0)
        except KeyboardInterrupt:
            break
    gcs.MOV1([1,2,3],[offset[0]+delta_dist,offset[1]+delta_dist*y_points,offset[2]])
                        
                    
        
def move_to_point(gcs, offset, trigger_channel = 1):
    """
        Move stage command to selected position in 3D and trigger signal after arrival  
        gcs : piezo stage object

        offset : start position of the stage ([float,float,float], range[0,200])
        trigger_channel : output trigger channel (int, in [1,2,3])
    """    
    if max(offset)>200 or min(offset)<0:
        print('Set correct position between(0,200)')
    else:
        gcs.VEL1([1,2,3],[10000,10000,10000])    
        gcs.CTO1(trigger_channel,[2,3],[1,2])
        gcs.MOV1([1,2,3],offset)
        
def set_trigger_params(trigger_output,window_range=1,time_range=0):
    """
    Set in memory parameters of on-target status parameters
    trigger output :number of output channel (1,2,3)
    window_range: distance between real value and target in which stage send high value
    time range: time for how long stage is in range to send a high value"""
    gcs.SPA1(trigger_output,'0x07000900',window_range )
    gcs.SPA1(trigger_output,'0x07000901',time_range )   


In [7]:
# Set on-target status parameters

gcs.CCL1(1,'advanced')
set_trigger_params(1,0.015,0.01)
set_trigger_params(2,0.015,0.01)
set_trigger_params(3,0.015,0.01)


In [8]:

# Process:
#1) Set on QT number of array elements 
# format (X:pixels in one line minus 1, Y :more than number of lines in scan)
#2) Tick "timestamping" and set number of timestamps(+10% than expected)
#3) Set upper start
#4) Run this cells
#5) name output file and save
#6) Immidiatly press enter in input window below. 
nanoSPAD_X=10
nanoSPAD_Y=10
point_by_point_snake_3(gcs,nanoSPAD_X+1,nanoSPAD_Y,dwell_time = 0.05,
                        delta_dist = 0.1,offset = [25.,25.,25.],rep=1,velocity=10)




0.07333333333333333
Press enter to start
0 0 0.009999990463256836
0 0 0.08800005912780762 0.0 0.04
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
1 10
1 9
1 8
1 7
1 6
1 5
1 4
1 3
1 2
1 1
1 0
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
3 10
3 9
3 8
3 7
3 6
3 5
3 4
3 3
3 2
3 1
3 0
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
5 10
5 9
5 8
5 7
5 6
5 5
5 4
5 3
5 2
5 1
5 0
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
7 10
7 9
7 8
7 7
7 6
7 5
7 4
7 3
7 2
7 1
7 0
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
9 10
9 9
9 8
9 7
9 6
9 5
9 4
9 3
9 2
9 1
9 0


In [23]:
move_to_point(gcs, [25.-.1,25.-.1,25.,])

In [50]:
gcs.CloseConnection()

In [10]:
vel =0.1*1e6/(2*50)
gcs.VEL1([1,2,3],[vel,vel,vel])

In [ ]:
def point_by_point_snake(gcs,x_points,y_points,dwell_time,
                       delta_dist,offset,rep = 10 ,trigger_channel = 1):
    """
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger on selected channel send signal when stage is on target
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
        rep : number of cycles of the movement (int, 0 means infinity, )
        trigger_channel : output trigger channel (int, in [1,2,3])
    """
    
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    gcs.MOV1([1,2,3],offset)
    time.sleep(max(dwell_time-0.01,0))
    gcs.CTO1(trigger_channel,[2,3],[1,2])
    print(0,0)
    x_pos=np.arange(offset[0],x_points*delta_dist,delta_dist)
    y_pos=np.arange(offset[1],y_points*delta_dist,delta_dist)
    for a in range(rep):
        try:
            for i in range(int(x_points/2)):
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])
                    time.sleep(max(dwell_time-0.01,0))
                    gcs.CTO1(trigger_channel,[2,3],[1,2])
                    print(2*i,j+1)
                gcs.CTO1(trigger_channel,[2,3],[2,2])
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1),offset[2]])
                time.sleep(max(dwell_time-0.01,0))
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                print((2*i+1),j+1)
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    time.sleep(max(dwell_time-0.01,0))
                    gcs.CTO1(trigger_channel,[2,3],[1,2])    
                    print(2*i+1,y_points-j-2)
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                time.sleep(max(dwell_time-0.01,0))
                gcs.CTO1(trigger_channel,[2,3],[1,2])
                print(2*i+2,0)
        except KeyboardInterrupt:
            break

##Correctting


In [8]:
def point_by_point_snake_2(gcs,x_points,y_points,dwell_time,
                       delta_dist,offset,rep = 10):
    """
        V1: Using trigger mode 0
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger signals reresent new point/line/frame(channels 1/2/3  respectively)
        
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
        rep : number of cycles of the movement (int)
    """
    #Important parameters
    half = dwell_time *1/2
    #Setting velocity of the piezo (required to be low enough for trigger step size)
    vel =delta_dist*1e6/(25)
    gcs.VEL1([1,2,3],[vel,vel,vel])
    #Snake start 
    start =time.time()
    #Set in point nerby starting position(for trigger)
    gcs.MOV1([1,2,3],[offset[0]-delta_dist/4.,offset[1]-delta_dist/4.,offset[2]--delta_dist/4.])
    #Set triggers
    gcs.CTO1(1,[2,3,1],[1,0,delta_dist])
    gcs.CTO1(2,[2,3,1],[2,0,delta_dist])
    gcs.CTO1(3,[2,3,1],[3,0,delta_dist])

    gcs.MOV1([1,2,3],[offset[0]-delta_dist/2.,offset[1]-delta_dist/2.,offset[2]-delta_dist/4.])
    t1=time.time()
    while (time.time()-t1<0.01):
        continue 
    for a in range(rep):
        try:
            #Move to initial point
            gcs.MOV1([1,2,3],offset)
            t1=time.time()
            while (time.time()-t1<dwell_time):
                continue
            tnow=time.time()
            #move to point between in purpose of trigger mode and stay for 10ms
            gcs.MOV1([1,2,3],[offset[0],offset[1]+half,offset[2]])
            t1=time.time()
            while (time.time()-t1<0.01):
                continue           
            for i in range(int(x_points/2)):
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<dwell_time):
                        continue                   
                    print(2*i,j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                    if j!=y_points-2:
                        
                        gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1)+half,offset[2]])
                        t1=time.time()
                        while (time.time()-t1<0.01):
                            continue 
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i+half,offset[1]+delta_dist*(y_points-1)+half,offset[2]])
                t1=time.time()
                while (time.time()-t1<0.01):
                    continue                             
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-1),offset[2]])
                t1=time.time()
                while (time.time()-t1<dwell_time):
                    continue  
                print((2*i+1),j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                tnow= time.time()
                
                
                for j in range(int(y_points)-1):
                    
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<dwell_time):
                        continue                  
                    print(2*i+1,y_points-j-2,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                    if j!=y_points-2:
                        
                        gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1)-half,offset[2]])
                        t1=time.time()
                        while (time.time()-t1<0.01):
                            continue

                if i==int(x_points/2)-1:
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i+half,offset[1]+delta_dist*(y_points-1)-half,offset[2]])
                    t1=time.time()
                    while (time.time()-t1<0.01):
                        continue
                        
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                    t1=time.time()
                    
                    while (time.time()-t1<dwell_time):
                        continue  
                    print((2*i+1),j+1,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
                    tnow= time.time()
                else:
                    gcs.MOV1([1,2,3],[offset[0]-half,offset[1]-half,offset[2]-half])
                    t1=time.time()
                    while (time.time()-t1<0.01):
                        continue 
        except KeyboardInterrupt:
            break

def point_by_point_snake_3(gcs,y_points,x_points,dwell_time,
                       delta_dist,offset,rep = 10 ,velocity=50):
    """
        V3 using trigger mode 2
        Move stage in 2D on rectangular area and stop on every point for set dwell time.
        Trigger on selected channel send signal when stage is on target
        Parameters:
        gcs : piezo stage object
        x_points,y_points : Number of measuring points in both axis (int, minimum 2)
        dwell_time : stop time for one point in seconds (float, minimum 0.01s)
        delta_dist : distance between two neighbor points in micrometers (float, minimum 0.01um)
        offset : start position of the stage ([float,float,float], range[0,200])
       velocity: velocity of the stage
       
       Output  is +-2% of registered time and may depend on velocity
    """
    wait_time =dwell_time +delta_dist*1.0/(velocity*1.2)+0.015
    print(wait_time)
    gcs.VEL1([1,2,3],[10000,10000,10000])
    gcs.MOV1([1,2,3],[offset[0]-delta_dist,offset[1],offset[2]-delta_dist])
    input('Press enter to start')
    gcs.VEL1([1,2,3],[velocity,velocity,velocity])
    start =time.time()
    gcs.CTO1(1,[2,3],[1,2])
    gcs.CTO1(2,[2,3],[2,2])
    gcs.CTO1(3,[2,3],[3,2])

    t1=time.time()
    print(0,0,t1-start)
    tnow= time.time()
    for a in range(rep):
        try:
            #Move to initial point
            gcs.MOV1([1,2,3],offset)
            t1=time.time()
            while (time.time()-t1<wait_time):
                continue
            tnow=time.time() 
            print(0,0,time.time()-start,time.time()-tnow,max(dwell_time-0.01,0))
            for i in range(int(x_points/2)):
                
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*2*i,offset[1]+delta_dist*(j+1),offset[2]])

                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time()
                    print(2*i,j+1)
                gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(j+1),offset[2]])
                t1=time.time()
                while (time.time()-t1<wait_time):
                    continue
                tnow=time.time() 
                print(2*i+1,j+1)
                for j in range(int(y_points)-1):
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+1),offset[1]+delta_dist*(y_points-j-2),offset[2]])
                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time() 
                    print(2*i+1,y_points-j-2)
                if i <int(x_points/2)-1:
                    gcs.MOV1([1,2,3],[offset[0]+delta_dist*(2*i+2),offset[1],offset[2]])
                    t1=time.time()
                    while (time.time()-t1<wait_time):
                        continue
                    tnow=time.time()
                    print(2*i+2,0)
        except KeyboardInterrupt:
            break
    gcs.MOV1([1,2,3],[offset[0]+delta_dist,offset[1]+delta_dist*y_points,offset[2]])
                        
        